In [1]:
from widgetscript import JsContext, PrecompiledJsContext, pycall
from IPython.display import HTML, display
from yattag import Doc

In [2]:
def get_calculator_context(precompiled_context = None):
    ctx = JsContext(precompiled_context = precompiled_context)
    
    from math import sqrt, log, exp, pi, e, tan, cos, sin, floor, ceil
    calculator_globals = {func.__name__: func for func in [sqrt, log, exp, tan, cos, sin, floor, ceil]}
    calculator_globals["e"] = e
    calculator_globals["pi"] = pi

    @ctx.py
    def eval_expression(expression):
        return eval(expression, calculator_globals)
    
    @ctx.js
    def on_enter(e):
        if e.keyCode == 13:
            e.preventDefault()
            expression = e.target.value
            pycall(eval_expression(expression), got_result, got_error)
            
    @ctx.js
    def got_result(result):
        document.getElementById("expression_result_"+__context_id).textContent = result
        
    @ctx.js
    def got_error(error):
        console.log(error)
        document.getElementById("expression_result_"+__context_id).textContent = "Error"
        
    @ctx.js_flat
    def js_init():
        expression_input = document.getElementById("expression_input_"+__context_id)
        expression_input.addEventListener("keyup", on_enter) 
        
    return ctx


In [3]:
get_calculator_context().compile().save("./calculator_context")
precompiled_context = PrecompiledJsContext.load("./calculator_context")

In [4]:
class CalculatorWidget:
    
    def __init__(self):
        ctx = get_calculator_context(precompiled_context)

        doc, tag, text = Doc().tagtext()

        with tag("div"): 
            doc.stag("input", id="expression_input_"+ctx.context_id, style="display: inline-block")
            with tag("span", style="margin-left: 7px; margin-right: 5px"):
                text("=")
            doc.stag("span", id="expression_result_"+ctx.context_id)
            
        doc.asis(ctx.html())
        
        self.html = doc.getvalue()
    
    def _repr_html_(self):
        return self.html  

In [5]:
CalculatorWidget()

<IPython.core.display.Javascript object>